TRABAJO GRUPAL PEL

Establecer los paths donde estan las librerias externas y cargarlas

In [1]:
#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libcurl")

#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libjsoncpp")

Incluir las librerias necesarias

In [2]:
#include <iostream>
#include <fstream>
#include <string>
#include <array>

//Curl
#include <curl/curl.h>

//Json
#include <json/json.h>
#include <json/reader.h>
#include <json/writer.h>
#include <json/value.h>

//Mapas
#include "xleaflet/xmap.hpp"
#include "xleaflet/xtile_layer.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xmarker_cluster.hpp"
#include "xleaflet/xicon.hpp"
#include "xleaflet/xbasemaps.hpp"
#include "xleaflet/xpopup.hpp"
#include "xwidgets/xhtml.hpp"

//mapas y json
#include "xleaflet/xgeo_json.hpp"

using namespace std;


Estructura para las gasolineras

In [3]:
struct nodo {
    string provincia;
    string municipio;
    string localidad;
    double longitud;
    double latitud;
    double precio;

    nodo *sgt;
};

In [4]:
nodo* head=NULL, *fin =NULL, *a=NULL;

In [5]:

void eQ(nodo *actual) {

    if(fin ==NULL){
        actual->sgt=NULL;
        fin =actual;
        head=actual;
    }else{
        fin ->sgt=actual;
        actual->sgt=NULL;
        fin =actual;
    }
}

Funcion para manejar la informacion de respuesta del GET REQUEST

In [6]:
static size_t WriteCallback(char* contenido, size_t size, size_t num, string* out) {
    out -> append(contenido, size * num);
        return size * num;
}

In [7]:
double toDouble(string s) {
    replace(s.begin(), s.end(), ',', '.');
        return atof(s.c_str());
}

In [8]:
static Json::Value ParseJson(string content) {
    Json::CharReaderBuilder builder;
    Json::CharReader* reader = builder.newCharReader();
    
    Json::Value jsonData;
    string errors;
    
    bool exitoParseando = reader->parse(content.c_str(), content.c_str() + content.size(), &jsonData, &errors);
    delete reader;
    
    if (!exitoParseando) {
        cout << "Error transfromando la informacion HTTP a JSON" << std::endl;
        cout << "Los datos del HTTP eran:\n" << content << std::endl;
        return 0;
    }
    else {
        cout << "Conversion de los datos a JSON exitosa" << endl;
        cout << "\ndatos del JSON recibidos" << endl;
        //cout << jsonData.toStyledString() << endl;

        const string dateString(jsonData["Fecha"].asString());


       std::cout << "\tDate string: " << jsonData["Fecha"] << std::endl;
        std::cout << std::endl;
    }
    return jsonData;
    
}

In [9]:
void getGasolineras()
{
    const std::string url("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/FiltroProvinciaProducto/28/1");
    CURL* curl = curl_easy_init();
    string readBuffer;
    long httpCode(0);
    
    if (curl) {
        // Configurar como request GET
        curl_easy_setopt(curl, CURLOPT_CUSTOMREQUEST, "GET");
        // Configurar el URL.
        curl_easy_setopt(curl, CURLOPT_URL, url.c_str());
        // Para que no intente IPv6, porque aumentaria el tiempo de resolucion de DNS.
        curl_easy_setopt(curl, CURLOPT_IPRESOLVE, CURL_IPRESOLVE_V4);
        //Para que solo espere maximo 10 segundos la respuesta
        curl_easy_setopt(curl, CURLOPT_TIMEOUT, 10);
        // Follow HTTP redirects if necessary.
        curl_easy_setopt(curl, CURLOPT_FOLLOWLOCATION, 1L);
        
        // Response information.
        
        
        // Enlazar la funcion que va a manejar la informacion de respuesta
        curl_easy_setopt(curl, CURLOPT_WRITEFUNCTION, WriteCallback);
        
        // Enlazar un contenedor para la informacion (will be passed as the last parameter to the
        // callback handling function).  
        curl_easy_setopt(curl, CURLOPT_WRITEDATA, &readBuffer);
        
        // Run our HTTP GET command, capture the HTTP response code, and clean up.
        curl_easy_perform(curl);
        curl_easy_getinfo(curl, CURLINFO_RESPONSE_CODE, &httpCode);
    }
   curl_easy_cleanup(curl);
    

    if (httpCode == 200)
    {
        std::cout << "\nGot successful response from " << url << std::endl;
        // La respuesta correcta - terminado con lo de curl.  
        // Intentar parsear los resultados e imprimirlos
        Json::StreamWriterBuilder builder;
        Json::Value jsonData = ParseJson(readBuffer);
        
        const Json::Value& listaprecio = jsonData["ListaEESSPrecio"];
        
        int i=0;
        while(i< listaprecio.size()) {
            nodo *gasolinera= new nodo();
            gasolinera->municipio = listaprecio[i]["Municipio"].asString();
            gasolinera->provincia = listaprecio[i]["Provincia"].asString();
            gasolinera->localidad = listaprecio[i]["Localidad"].asString();
            gasolinera->latitud = toDouble(listaprecio[i]["Latitud"].asString());
            gasolinera->longitud = toDouble(listaprecio[i]["Longitud (WGS84)"].asString());
            gasolinera->precio = toDouble(listaprecio[i]["PrecioProducto"].asString());
            //cout << "Latitud: " << toDouble(listaprecio[i]["Latitud (WGS84)"].asString()) <<endl;
            eQ(gasolinera);
            i++;
        }
        cout << "Numero de gasolineras: " << i << endl;
    }
    else
    {
        std::cout << "Couldn't GET from " << url << " - exiting" << std::endl;
        return 1;
    }

    return 0;
}

In [10]:
getGasolineras();


Got successful response from https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/FiltroProvinciaProducto/28/1
Conversion de los datos a JSON exitosa

datos del JSON recibidos
	Date string: "13/01/2021 4:33:30"

Numero de gasolineras: 739


In [11]:
//Para hacer una cola de todos las coincidencias de la busqueda
void push_nodo(nodo* coincidencia, nodo* &peek){
    nodo *aux = new nodo;
    aux->municipio= coincidencia->municipio;
    aux->provincia=coincidencia->provincia;
    aux->latitud=coincidencia->latitud;
    aux->longitud=coincidencia->longitud;

    aux->sgt=peek;
    peek=aux;
}

In [12]:
//Para hacer busqeuda por municipio y provincia
nodo* localizaciones(string provincia){
    nodo* aux = head;
    //voy a crear una nueva cola para guardar los resultados
    nodo* peek=NULL;

    if (head == NULL)
        cout << "Lista vacia, nada que buscar";
    else {
        while (aux != NULL) {
            if (provincia == aux->provincia) {
                push_nodo(aux,peek);
            }
            aux = aux->sgt;
        }

    }
    return peek;
}

In [13]:
//Queria utilizar tambien Stacks para practicar y porque son muy practicas
string prtresultados(nodo* peek){
    string contenido="Nombre \t\t Apellido\t\t Expediente\n";
    nodo *aux = peek;
    int i=0;
    if(aux==NULL)
        cout<<"Nada que imprimir, Stack vacia\n";
    else {
        //cout << contenido << endl;
        while (aux != NULL && i<5) {
            cout << "Nombre: " << aux->municipio << " " << aux->provincia << "\nLatitud: " << aux->latitud
                 << "\n" << endl;
            contenido+=aux->municipio + "\t\t" + aux->provincia + "\t\t\t" + to_string(aux->latitud) + "\n";
            aux = aux->sgt;
            i++;
        }
    }
    return contenido;
}

In [14]:
prtresultados(head);

Nombre: Ajalvir MADRID
Latitud: 40.5267

Nombre: Álamo (El) MADRID
Latitud: 40.2515

Nombre: Álamo (El) MADRID
Latitud: 40.2391

Nombre: Alcalá de Henares MADRID
Latitud: 40.4982

Nombre: Alcalá de Henares MADRID
Latitud: 40.5083



MAPA

In [9]:
xlf::map mymap;
nodo *aux = localizaciones("MADRID", "MADRID");
mymap.zoom = 10;
// latitude 52.204793, longitude 360.121558
mymap.center = std::array<double, 2>({aux->latitud, aux->longitud});
mymap.display();

A Jupyter widget

In [10]:
auto marker1 = xlf::marker::initialize()
    .location({(aux->latitud)+0.01, (aux->longitud)+0.01})
    .title(aux->municipio)
    .finalize();

In [11]:
mymap.add_layer(marker1);

mymap.display()

A Jupyter widget

In [12]:
xw::html message1;
xw::html message2;
message1.value = "Try clicking the marker!";
message2.value = "Hello <b>World</b>";
message2.placeholder = "Some HTML";
message2.description = "Some HTML";

// Popup with a given location on the map:
auto popup = xlf::popup::initialize()
    .location({aux->latitud, aux->longitud})
    .child(message1)
    .close_button(false)
    .auto_close(false)
    .close_on_escape_key(false)
    .finalize();
mymap.add_layer(popup);

// Popup associated to a layer
marker1.popup = message2;

In [ ]:
nodo *aux = localizaciones("MADRID", "MADRID");
xlf::marker marker;
int i=0;
array<xlf::marker, 5> array_markers;
    
        //cout << contenido << endl;
        while (aux != NULL && i<5) {
             std::cout << "en el while" <<std::endl;
             marker.location=std::array<double, 2>({aux->latitud, aux->longitud});
            array_markers[i] = marker;
            aux = aux->sgt;
            i++;
        }
    cout << array_markers.size() << endl;

xlf::marker_cluster marker_cluster = xlf::marker_cluster::initialize() .markers(array_markers)
.finalize();

mymap.add_layer(marker_cluster);

In [ ]:
nodo *aux = localizaciones("MADRID", "MADRID");
xlf::marker marker;
int i=0;
array<xlf::marker, 5> array_markers;
    
        //cout << contenido << endl;
        while (aux != NULL && i<5) {
             std::cout << "en el while" <<std::endl;
             marker.location=std::array<double, 2>({aux->latitud, aux->longitud});
            array_markers[i] = marker;
            aux = aux->sgt;
            i++;
        }
    cout << array_markers.size() << endl;

xlf::marker_cluster marker_cluster = xlf::marker_cluster::initialize() .markers(array_markers)
.finalize();

mymap.add_layer(marker_cluster);

In file included from input_line_9:2:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xhtml.hpp:13:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xstring.hpp:14:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xwidget.hpp:15:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xlayout.hpp:16:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xobject.hpp:18:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xtransport.hpp:19:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xproperty/xobserved.hpp:20:
/Users/sol/opt/miniconda3/envs/cling/include/xproperty/xproperty.hpp:201:19: error: no matching conversion for functional-style cast from
      'std::__1::array<xw::xmaterialize<xlf::xmarker>, 5>' to
      'xp::xproperty<std::__1::vector<xw::xholder,
      std::__1::allocator<xw::xholder> >, xw::xmaterialize<xlf:

In [44]:
mymap.add_layer()

A Jupyter widget